# PAWPULARITY PETFINDER INFERENCE

### [Training Notebook](https://www.kaggle.com/tarunbisht11/find-a-pet-with-lightning-speed-effnets)

### Internet Disabled
Make sure to disable internet for submitting predictions

### What's New
- tta added
- models and augmentations updated

In [ ]:
! pip install -qq ../input/timm-pytorch-image-models/pytorch-image-models-master

In [ ]:
import os
import gc
import random
import time
import math

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import torchmetrics as metrics
import timm

from kaggle_datasets import KaggleDatasets

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class Config:
    '''general'''
    num_workers = 4
    
    '''data'''
    batch_size = 64
    target_col = "Pawpularity"
    
    '''model'''
    # 'regnety_002', 'efficientnet_b3', 'efficientnet_b0', 
    # 'vit_base_patch16_224', 'tf_efficientnet_b4_ns'
    model_name = 'tf_efficientnet_b3'
    # number of predictors
    targets = 1
    # regressor features, number of useful features in meta data
    # 12 means using all columns except id and target column
    num_features = 12
    # input image size send to network can be like 256, 512, 768, 1028
    input_size = 260
    # freeze backbone network
    freeze_backbone = True
    
    '''test'''
    n_fold = 5
    trn_folds = [0, 1, 2, 3, 4]
    tta_steps = 10

In [ ]:
test_csv = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
test_data_path = "../input/petfinder-pawpularity-score/test"
test_csv["path"] = test_csv["Id"].apply(lambda x: os.path.join(test_data_path, x+".jpg"))
test_csv.head()

In [ ]:
max_paws = 100

In [ ]:
class PawpularityDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, augmentations=None, test=False):
        super(PawpularityDataset, self).__init__()
        self.csv = csv_file
        self.test = test
        self.augs = augmentations
        self.length = len(self.csv)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        path = self.csv.iloc[idx]["path"]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.augs is not None:
            img = self.augs(image=img)['image']
        else:
            img = torch.from_numpy(img).float()
            img = img.permute(2, 0, 1)
        meta = torch.from_numpy(self.csv.iloc[idx, 1:13].values.astype(np.float32))
        if self.test:
            return img, meta
        label = torch.tensor(self.csv.iloc[idx][Config.target_col], dtype=torch.float)/max_paws
        return img, meta, label

In [ ]:
class ImageAugmentations:
    '''
        image_size: resize image to -> (width, height)
        train_augs: include augmentations like random crop, rotation etc training if false then return
                    only resize image as pytorch tensor
    '''
    def __init__(self, image_size, apply_augs=False):
        self.image_size = image_size
        self.apply_augs = apply_augs
        
    def train_augs(self):
        if self.apply_augs:
            return A.Compose([A.Resize(self.image_size, self.image_size),
                              A.HorizontalFlip(p=.5),
                              A.ChannelShuffle(p=.1),
                              A.ColorJitter(brightness=0.1,
                                            hue=0.1,
                                            saturation=0.1,
                                            contrast=0.1,
                                            p=.2),
                              A.RandomGamma(p=.1),
                              A.Sharpen(p=.1),
                              A.Cutout(p=.2),
                              # imagenet normalization
                              A.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],
                                          max_pixel_value=255.0,
                                          p=1.0),
                              ToTensorV2()])
        return A.Compose([A.Resize(self.image_size, self.image_size),
                          A.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225],
                                      max_pixel_value=255.0,
                                      p=1.0),
                          ToTensorV2()])
    
    def valid_augs(self):
        return A.Compose([A.Resize(self.image_size, self.image_size),
                          A.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225],
                                      max_pixel_value=255.0,
                                      p=1.0),
                          ToTensorV2()])
        

In [ ]:
augs = ImageAugmentations(Config.input_size, apply_augs=True)
ds = PawpularityDataset(test_csv, augmentations=augs.train_augs(), test=True)
loader = torch.utils.data.DataLoader(ds,
                                     shuffle=True,
                                     batch_size=4)
images, meta = next(iter(loader))
plt.figure(figsize=(15, 15))
for step, image in enumerate(images):
    plt.subplot(1, 4, step+1)
    plt.imshow(image.permute(1, 2, 0).numpy())
    plt.axis('off')

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super(PawpularityModel, self).__init__()
        self.cfg = cfg
        timm_model = timm.create_model(self.cfg.model_name, 
                                       pretrained=pretrained, 
                                       in_chans=3)
        
        if self.cfg.freeze_backbone:
            modules = []
            for module in timm_model.children():
                for param in module.parameters():
                    param.requires_grad = False
                modules.append(module)
        else:
            modules = list(timm_model.children())
        
        self.classifier = modules[-1]
        cnn_out_features = self.classifier.in_features
        classifier_out_features = self.classifier.out_features
        
        self.cnn = nn.Sequential(*modules[:-1])
        self.dropout = nn.Dropout(0.2)
        
        self.meta_reg = nn.Linear(cfg.num_features, cfg.num_features)
        num_features = classifier_out_features + cfg.num_features + cnn_out_features
        self.regressor_1 = nn.Sequential(nn.Linear(num_features, 128),
                                         nn.SiLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(128, 64),
                                         nn.SiLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(64, 32),
                                         nn.SiLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(32, 16),
                                         nn.SiLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(16, cfg.targets))
        num_features = cnn_out_features + cfg.num_features
        self.regressor_2 = nn.Sequential(nn.Linear(num_features, 64),
                                         nn.SiLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(64, cfg.targets))
        
    def forward(self, img, meta):
        meta = self.meta_reg(meta)
        cnn_features = self.cnn(img)
        if self.cfg.freeze_backbone:
            classes = self.classifier(cnn_features)
            cnn_features = self.dropout(cnn_features)
            x = torch.cat((cnn_features, meta, classes), dim=1)
            x = self.dropout(x)
            return self.regressor_1(x)
        cnn_features = self.dropout(cnn_features)
        x = torch.cat((cnn_features, meta), dim=1)
        x = self.dropout(x)
        return self.regressor_2(x)

In [ ]:
class LitPawpularity(pl.LightningModule):
    def __init__(self, cfg, model):
        super(LitPawpularity, self).__init__()
        self.cfg = cfg
        self.model = model
        
    def forward(self, img, meta):
        return self.model(img, meta)
        
    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        img, meta = batch
        out = self(img, meta).flatten()
        return F.sigmoid(out) * max_paws

In [ ]:
def get_weights_path(fold, mode):
    path = "../input/find-a-pet-with-lightning-speed-effnets"
    return os.path.join(path, "best", f"{Config.model_name}_best_{mode}_fold{fold}.ckpt")

In [ ]:
def get_fold_predictions(fold):
    print("="*10)
    print("Predictions using fold", fold)
    print("="*10)
    augs = ImageAugmentations(Config.input_size, apply_augs=True)
    weights = [get_weights_path(fold, "loss"),
               get_weights_path(fold, "rmse")]
    preds = []
    for weight in weights:
        print("Using weights: ", weight)
        model = PawpularityModel(Config, pretrained=False)
        lit = LitPawpularity.load_from_checkpoint(weight, cfg=Config, model=model, fold=fold)
        trainer_params = {"gpus": -1}
        trainer = pl.Trainer(**trainer_params)
        tta_preds = []
        for tta in range(Config.tta_steps):
            if tta == 0:
                ds = PawpularityDataset(test_csv,
                                        augmentations=augs.valid_augs(),
                                        test=True)
            else:
                ds = PawpularityDataset(test_csv,
                                        augmentations=augs.train_augs(),
                                        test=True)
            loader = torch.utils.data.DataLoader(ds,
                                                 shuffle=False,
                                                 num_workers=Config.num_workers, 
                                                 pin_memory=True,
                                                 drop_last=False,
                                                 batch_size=Config.batch_size*2)
            predictions = trainer.predict(lit, loader)
            tta_preds.append(torch.cat([x for x in predictions]).detach().cpu().numpy())
        predictions = np.mean(np.column_stack(tta_preds), axis=1)
        preds.append(predictions)
    preds = np.mean(np.column_stack(preds), axis=1)
    return preds

In [ ]:
fold_predictions = []
for fold in Config.trn_folds:
    preds = get_fold_predictions(fold)
    fold_predictions.append(preds)
    
fold_predictions = np.column_stack(fold_predictions)

In [ ]:
mean_predictions = np.mean(fold_predictions, axis=1)

In [ ]:
test_csv["Pawpularity"] = mean_predictions
sub_csv = test_csv[["Id", "Pawpularity"]]
sub_csv.to_csv("submission.csv", index=False)
sub_csv.head()